# HW3 by Linsen Li

# 1. N-gram model

## (a) Preprocess the train and validation data, build the vocabulary, tokenize

In [1]:
with open('./a3-data/train.txt', "r") as f:
        train_text = [line for line in f]

with open('./a3-data/valid.txt', "r") as f:
        test_text = [line for line in f]
        
import nltk
train_sentence = ' '.join(train_text).replace('<unk>', '')
test_sentence = ' '.join(test_text).replace('<unk>', '')
train_tokens = nltk.word_tokenize(train_sentence)
test_tokens = nltk.word_tokenize(test_sentence)

In [2]:
# Built the vocabulary
from nltk.lm import Vocabulary
vocab1 = Vocabulary(train_tokens, unk_cutoff=2)
vocab2 = Vocabulary(test_tokens, unk_cutoff=2)

In [3]:
len(vocab1)

9962

In [4]:
len(vocab2)

3984

In [5]:
# Tokenize the text.
import re
from nltk import word_tokenize
from nltk.lm.preprocessing import pad_both_ends
from nltk.util import everygrams
from nltk.lm.preprocessing import padded_everygram_pipeline

sent_tokenize = lambda x: re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', x)
train_tokenized_text = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(train_sentence)]
test_tokenized_text = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(test_sentence)]
# Preprocess the tokenized text for 3-grams language modelling
n = 3
train, v_train = padded_everygram_pipeline(3, train_tokenized_text)
test, v_test = padded_everygram_pipeline(3, test_tokenized_text)

## (b) Implement an N-gram model (trigram) for language modeling

In [173]:
# Bulid the trigram model
from nltk.lm import MLE
ngram = MLE(3) 

In [174]:
# Before training, the vocab of model is 0
len(ngram.vocab)

0

In [175]:
# Train the trigram model
ngram.fit(train, v_train)

In [176]:
# After training the n-gram model, the vocab of model
len(ngram.vocab)

10000

In [10]:
print(ngram.vocab.lookup(train_tokenized_text[0]))

('aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack-food', 'ssangyong', 'swapo', 'wachter', 'pierre', 'n', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov', '.')


In [178]:
# Show the n-gram model
print(ngram.counts)

<NgramCounter with 3 ngram orders and 2544687 ngrams>


In [290]:
# Check some probability
ngram.score('is', 'this'.split())  # P('is'|'this')

0.08285479901558655

In [298]:
# Check some probability
ngram.score('of', 'the type'.split())  # P('of'|'the type')

0.7857142857142857

## (c) Implement Good Turing smoothing

In [6]:
# Use Good Turing estimation
from nltk import SimpleGoodTuringProbDist,FreqDist 
import nltk
ngrams = nltk.trigrams(train_sentence)
freq_dist = nltk.FreqDist(ngrams)
Good_turing = SimpleGoodTuringProbDist(freq_dist)

In [7]:
from nltk.model.ngram import NgramModel
est = lambda freq_dist: SimpleGoodTuringProbDist(freq_dist)
gt = NgramModel(3, corpus, estimator=est)

<SimpleGoodTuringProbDist based on 4918583 samples>


## (d) Implement Kneser-Ney Smoothing

In [400]:
# Use kneser_ney estimation
import nltk
ngrams = nltk.trigrams(train_sentence)
freq_dist = nltk.FreqDist(ngrams)
kneser_ney = nltk.KneserNeyProbDist(freq_dist)

In [391]:
print(kneser_ney)

<KneserNeyProbDist based on 4918583 trigrams


In [389]:
# Use nltk package to implement Kneser-Ney Smoothing
from nltk.lm.models import KneserNeyInterpolated,Lidstone
from nltk.lm.smoothing import KneserNey, WittenBell
from nltk.lm.api import Smoothing
kn = KneserNeyInterpolated(3)
kn.fit(train, v_train)

In [336]:
# After training the n-gram model, the vocab of model
len(kn.vocab)

10000

## (e) Predict the next word in the valid set using a sliding window. Report the perplexity scores of N-gram, Good Turing, and Kneser-Ney on the test set

In [263]:
# Create a sliding window on valid set
# Set the window size as 20
# So we are using previous 19 words to predict the 20th word
import numpy as np
windows = list()
for i in range(19, len(test_tokens)):
    window = test_tokens[i-19: i+1]
    windows.append(window)
previous_19 = [i[:-1] for i in windows]
actual_20 = [''.join(i[-1:]) for i in windows]
# Predict the 20th word basing on the previous 19 word
# All prediction are stored in the list predict_word
predict_word = [ngram.generate(1,text_seed=i, random_seed=3) for i in previous_19]

In [404]:
# Show some predict result 
# We only show first 10 predict in this case
print('The predict word: ')
print(predict_word[:10])
print('\nThe actual word in test set:')
print(actual_20[:10])

The predict word: 
['football', 'explains', 'be', 'bring', 'for', 'market', 'independence', 'few', 'contribution', 'between']

The actual word in test set:
['football', 'can', 'now', 'vote', 'during', 'for', 'the', 'greatest', 'play', 'in']


In [284]:
# Calculate the accuracy
num = 0
for i in range(len(predict_word)):
    if actual_20[i] == predict_word[i]:
        num += 1
print('The accuracy of Predicting the next word in sliding window : %f' % (num/len(predict_word)))

The accuracy of Predicting the next word in sliding window : 0.096465


In [375]:
# To calculate the perplexity scores on the test set
p_ngram = np.array([ngram.perplexity(i) for i in test_tokens])
per_ngram = np.ma.masked_invalid(p).mean()
print('The perplexity scores of ngram model on test set is :%f' % per_ngram)

The perplexity scores of ngram model on test set is :27576.069826


## (f) Choose the first 30 lines and print the predictions of next words using N-gram model

In [199]:
# Print the predictions of next words in the same 30 lines of input.txt as in N-gram.
# We first print out the sentence we want to predict
with open('./a3-data/input.txt', "r") as f:
        prediction_text = [line for line in f]
top_30 = prediction_text[0:30]
top_30

["but while the new york stock exchange did n't fall ___\n",
 'some circuit breakers installed after the october N crash failed ___\n',
 'the N stock specialist firms on the big board floor ___\n',
 'big investment banks refused to step up to the plate ___\n',
 "heavy selling of standard & poor 's 500-stock index futures ___\n",
 'seven big board stocks ual amr bankamerica walt disney capital ___\n',
 'once again the specialists were not able to handle the ___\n',
 '<unk> james <unk> chairman of specialists henderson brothers inc. it ___\n',
 'when the dollar is in a <unk> even central banks ___\n',
 'speculators are calling for a degree of liquidity that is ___\n',
 'many money managers and some traders had already left their ___\n',
 'then in a <unk> plunge the dow jones industrials in ___\n',
 '<unk> trading accelerated to N million shares a record for ___\n',
 'at the end of the day N million shares were ___\n',
 "the dow 's decline was second in point terms only ___\n",
 "in perce

In [236]:
from nltk import word_tokenize
def ngram_predict(text):
    # Tokenize the test to a list 
    # exclude the last word which we are to predict
    token = word_tokenize(text)[:-1]
    predict_word = ngram.generate(1,text_seed=token, random_seed=3)
    result = text.rstrip() + ': ' + predict_word
    print(result)

In [237]:
# Print out the prediction of next word using n-gram model
for i in range(30):
    print('\nExample %d:' % (i+1))
    ngram_predict(top_30[i])


Example 1:
but while the new york stock exchange did n't fall ___: by

Example 2:
some circuit breakers installed after the october N crash failed ___: labor-management

Example 3:
the N stock specialist firms on the big board floor ___: at

Example 4:
big investment banks refused to step up to the plate ___: and

Example 5:
heavy selling of standard & poor 's 500-stock index futures ___: contract

Example 6:
seven big board stocks ual amr bankamerica walt disney capital ___: for

Example 7:
once again the specialists were not able to handle the ___: abortion

Example 8:
<unk> james <unk> chairman of specialists henderson brothers inc. it ___: 's

Example 9:
when the dollar is in a <unk> even central banks ___: notably

Example 10:
speculators are calling for a degree of liquidity that is ___: creating

Example 11:
many money managers and some traders had already left their ___: demand

Example 12:
then in a <unk> plunge the dow jones industrials in ___: europe

Example 13:
<unk> trad

# 2. RNN model

In [46]:
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences

## (a),(b),(c),(d) 

In [89]:
# Data pre-processing
# In order to use RNN model to predict the next word
# Use integer encode text
# First generate training set

tokenizer = Tokenizer()
tokenizer.fit_on_texts([train_sentence])
encoded = tokenizer.texts_to_sequences([train_sentence])[0]
# Determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

# Set the window size as 20
sequences = list()
for i in range(19, len(encoded)):
    sequence = encoded[i-19:i+1]
    sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Window size: %d' % max_length)
# split into input and output elements
sequences = array(sequences)
X_train, y = sequences[:,:-1],sequences[:,-1]
y_train = to_categorical(y, num_classes=vocab_size)

Vocabulary Size: 9649
Total Sequences: 842605
Window size: 20


In [90]:
# Generate the test set
# Use the same approach above

tokenizer.fit_on_texts([test_sentence])
encoded_2 = tokenizer.texts_to_sequences([test_sentence])[0]
# determine the vocabulary size
vocab_size_2 = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size_2)

# Set the window of 20
sequences = list()
for i in range(19, len(encoded_2)):
    sequence = encoded_2[i-19:i+1]
    sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Window size: %d' % max_length)
# split into input and output elements
sequences = array(sequences)
X_test, y = sequences[:,:-1],sequences[:,-1]
y_test = to_categorical(y, num_classes=vocab_size)

Vocabulary Size: 9649
Total Sequences: 66834
Window size: 20


In [45]:
# Print out the shape of each part 
print('shape of X_train: ' + str(X_train.shape))
print('shape of y_train: ' + str(y_train.shape))
print('shape of X_test: ' + str(X_test.shape))
print('shape of y_test: ' + str(y_test.shape))

shape of X_train: (842605, 19)
shape of y_train: (842605, 9649)
shape of X_test: (66834, 19)
shape of y_test: (66834, 9649)


In [50]:
# Set up the structure of RNN model
from keras.layers import Dense, Dropout, BatchNormalization
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(60))
model.add(Dropout(0.5))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 19, 10)            96490     
_________________________________________________________________
lstm_2 (LSTM)                (None, 60)                17040     
_________________________________________________________________
dropout_2 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 9649)              588589    
Total params: 702,119
Trainable params: 702,119
Non-trainable params: 0
_________________________________________________________________
None


In [51]:
# Calcualte perplexity
import keras.backend as kb
def perplexity(y_predict, y_label):
    ce = kb.categorical_crossentropy(y_predict, y_label)
    return kb.exp(ce)

In [54]:
# Compile network
# Set up the training step: use a learning rate of 1e − 3 and an Adam optimizer
# Use perplexity as the metrics
# Also calculate the cross-entropy using categorical_crossentropy
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=[perplexity])
# Fit network
model.fit(X_train, y_train, epochs=50, verbose=1,batch_size = 50)

Epoch 1/50
842605/842605 [==============================] - 483s 573us/step - loss: 5.6951 - perplexity: 93405.6094
Epoch 2/50
842605/842605 [==============================] - 443s 526us/step - loss: 5.6126 - perplexity: 151171.9688
Epoch 3/50
842605/842605 [==============================] - 423s 502us/step - loss: 5.5860 - perplexity: 51704.7227
Epoch 4/50
842605/842605 [==============================] - 418s 496us/step - loss: 5.5723 - perplexity: 116320.7734
Epoch 5/50
842605/842605 [==============================] - 424s 503us/step - loss: 5.5665 - perplexity: 44029.1289
Epoch 6/50
842605/842605 [==============================] - 409s 485us/step - loss: 5.5612 - perplexity: 348676.7500
Epoch 7/50
842605/842605 [==============================] - 409s 486us/step - loss: 5.5558 - perplexity: 183300.5469
Epoch 8/50
842605/842605 [==============================] - 409s 485us/step - loss: 5.5518 - perplexity: 785979.1875
Epoch 9/50
842605/842605 [==============================] - 409s 48

In [74]:
import os
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
# This commented part is to load the existing RNN model
'''
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
'''

## (e) Train your RNN model. Calcuate the model’s perplexity on the test set

In [81]:
# Evaluate the model
# Calcuate the model’s perplexity on the test set
loss_and_acc = model.evaluate(X_test, y_test)
loss = loss_and_acc[0]
perplexity_test = loss_and_acc[1]
print('loss = ' + str(loss))
print('perplexity = ' + str(loss_and_acc[1]))

66834/66834 [==============================] - 13s 195us/step
loss = 5.891780329160139
perplexity = 12316.076171875


In [80]:
# Prove the perplexity equation
import numpy as np
sample_size = 66834
batch_size = 50
number_of_predictions = 837

total_loss = loss*sample_size/batch_size
aim = np.exp(total_loss/number_of_predictions)
print(aim)

12199.006990557798


As a result, we find that 
- perplexity = exp(total_loss/number_of_predictions)

## (f) Print the predictions of next words in the same 30 lines of input.txt

In [82]:
with open('./a3-data/input.txt', "r") as f:
        prediction_text = [line for line in f]

In [113]:
# Print the predictions of next words in the same 30 lines of input.txt as in N-gram.
# We first print out the sentence we want to predict
top_30 = prediction_text[0:30]
top_30

["but while the new york stock exchange did n't fall ___\n",
 'some circuit breakers installed after the october N crash failed ___\n',
 'the N stock specialist firms on the big board floor ___\n',
 'big investment banks refused to step up to the plate ___\n',
 "heavy selling of standard & poor 's 500-stock index futures ___\n",
 'seven big board stocks ual amr bankamerica walt disney capital ___\n',
 'once again the specialists were not able to handle the ___\n',
 '<unk> james <unk> chairman of specialists henderson brothers inc. it ___\n',
 'when the dollar is in a <unk> even central banks ___\n',
 'speculators are calling for a degree of liquidity that is ___\n',
 'many money managers and some traders had already left their ___\n',
 'then in a <unk> plunge the dow jones industrials in ___\n',
 '<unk> trading accelerated to N million shares a record for ___\n',
 'at the end of the day N million shares were ___\n',
 "the dow 's decline was second in point terms only ___\n",
 "in perce

In [238]:
# Given a sequence of words, predict the next word using RNN
def rnn_predict(text):
    encoded = tokenizer.texts_to_sequences([text])[0]
    # pre-pad sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=max_length-1, padding='pre')
    # predict probabilities for each word
    y_predict_class = model.predict_classes(encoded, verbose=0)
    # map predicted word index to word
    predict_word = ''
    for word, index in tokenizer.word_index.items():
        if index == y_predict_class:
            predict_word = word
            break
    # append to input
    result = text.rstrip() + ': ' + predict_word
    print(result)


In [239]:
# Print the predictions of next words in the same 30 lines of input.txt as in N-gram.
for i in range(30):
    text = top_30[i]
    print('\nExample ' + str(i+1) + ':')
    rnn_predict(text)



Example 1:
but while the new york stock exchange did n't fall ___: the

Example 2:
some circuit breakers installed after the october N crash failed ___: the

Example 3:
the N stock specialist firms on the big board floor ___: 's

Example 4:
big investment banks refused to step up to the plate ___: of

Example 5:
heavy selling of standard & poor 's 500-stock index futures ___: the

Example 6:
seven big board stocks ual amr bankamerica walt disney capital ___: to

Example 7:
once again the specialists were not able to handle the ___: company

Example 8:
<unk> james <unk> chairman of specialists henderson brothers inc. it ___: 's

Example 9:
when the dollar is in a <unk> even central banks ___: of

Example 10:
speculators are calling for a degree of liquidity that is ___: the

Example 11:
many money managers and some traders had already left their ___: the

Example 12:
then in a <unk> plunge the dow jones industrials in ___: the

Example 13:
<unk> trading accelerated to N million shares 